In [ ]:
import tensorflow as tf # Default graph is initialized when the library is imported
import os
from tensorflow.python.platform import gfile
from PIL import Image
import numpy as np
import scipy
from scipy import misc
import matplotlib.pyplot as plt
import cv2
from time import time

In [ ]:
proto_path = '/root/data/models/keypoints_detection/2019_05_02_17_19_32/weights/test.pb'

looks at this too 
* https://github.com/MarvinTeichmann/KittiSeg/blob/master/demo.py

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Solution 0

In [ ]:
with tf.Graph().as_default() as graph: # Set default graph as graph

    with tf.Session() as sess:
        # Load the graph in graph_def
        print("load graph")

        # We load the protobuf file from the disk and parse it to retrive the unserialized graph_drf
        with gfile.FastGFile(proto_path,'rb') as f:

            print("Load Image...")
            # Read the image & get statstics
            image = cv2.imread('/root/data/bati/images/23/4/2019-04-16/left_frame_crop_1006_793_2674_13851555410095.jpg')
            image = cv2.resize(image, (512, 512))
            image = np.expand_dims(image, axis=0)
            # image = image.astype(float)
            Input_image_shape=image.shape
            # height,width,channels = Input_image_shape

            print("Plot image...")
            #scipy.misc.imshow(image)

            # Set FCN graph to the default graph
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            sess.graph.as_default()

            # Import a graph_def into the current default Graph (In this case, the weights are (typically) embedded in the graph)

            tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="",
            op_dict=None,
            producer_op_list=None
            )

            # Print the name of operations in the session
#             for op in graph.get_operations():
#                     print("Operation Name :",op.name)         # Operation name
#                     print("Tensor Stats :",str(op.values()))     # Tensor name

            # INFERENCE Here
            l_input = graph.get_tensor_by_name('input_placeholder:0') # Input Tensor
            l_output = graph.get_tensor_by_name('stage_3/mid_conv7/BiasAdd:0') # Output Tensor

            print("Shape of input : ", tf.shape(l_input))
            #initialize_all_variables
            tf.global_variables_initializer()

            # Run Kitty model on single image
            start = time()
            Session_out = sess.run( l_output, feed_dict = {l_input : image} )
            end = time()
            print('Inference speed: {}'.format(end-start))

In [ ]:
image.shape

## Solution 1

In [ ]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="",
            op_dict=None,
            producer_op_list=None
            )
    return graph

In [ ]:
graph = load_graph(proto_path)
l_input = graph.get_tensor_by_name('input_placeholder:0') # Input Tensor
l_output = graph.get_tensor_by_name('stage_3/mid_conv7/BiasAdd:0') # Output Tensor

In [ ]:
for op in graph.get_operations():
    print("Tensor Stats :",str(op.values()))
    break# Tensor name

In [ ]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer()
    Session_out = sess.run( l_output, feed_dict = {l_input : image} )
    print(Session_out.shape)